In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator().build(number, verbose=False)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count()*5, number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(1000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

/home/luca/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
y_pred = model._model.predict(x_test)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9988621728428976, 0.9996193190501738)

In [7]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe()

In [8]:
X

,CodiceFiscale,CodiceCatasto,Document,Plate,Address,CAP,ProvinceCode,Region,Municipality,Year,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId
0,07488771218,,contratto,GA 123 KW,Via Del Riccio,70018,MI,,MONTORO INFERIORE,2024,...,Federico,Pastorino,voo,pamella.schmierer@schmierer.org,+39-397-555-403,,1972-04-23,F,0,41140
1,02858660232,A821,,GA 123KW,Via 4 Novembre,70027,PT,,APICE,,...,Renata,,opposto,chaya@malvin.com,+39-310-555-1208,"80.637,00 €",1978-03-05,F,1,
2,MRURCC77S22G154I,L407,verbali,GA123 KW,Corso Alberto Pio,26040,MS,LIGURIA,RIPE SAN GINESIO,,...,Rocco,Mura,nero,deandrea@yahoo.com,+39-371-055-504980,"55.793,00 €",1935-04-01,,,41142
3,01561040245,G151,pagamento,ZB 199AS,,98066,BL,CALABRIA,OGGEBBIO,2024.0,...,Simona,,il reparto latticini,estrella@aol.com,+39-373-555-455,,1966-01-10,donna,1,41143
4,,E250,fattura,,Via A. Nielli,,BO,,GALLO MATESE,2023,...,Riccardo,Paolicelli,adesso,gberray@gmail.com,+39-311-055-5565,"50.516,00 €",,maschio,,41144
5,03072570785,E423,accertamento,ZB 199AS,Via La Masa,27020,SO,SICILIA,,2001,...,Gianfranca,,posso aiutarti?,cory.gibes@gmail.com,+39-385-555-789,"26.791,00 €",,uomo,False,41145
6,00938080348,G303,verbali,,Borgo Giuseppe Garibaldi,28877,RA,,SANTA MARIA NUOVA,2025,...,Delfina Vittoria,Leonardi,il cognato,lemuel.latzke@gmail.com,,"85.440,00 €",,,SI,41146
7,SCRNMR56P65I040V,A780,ingiunzione,,Via Nazionale Modica-ispica,46041,RC,MOLISE,,2026,...,Anna Maria,Scarpa,biondo,paris@hotmail.com,+39-324-555-868,,1986-03-27,,0,
8,PSCRSO89S47M092B,F176,,ZB 199AS,Via Col De Lys,,,TOSCANA,BELLANTE,,...,Rosa,Piscitelli,,nicolette_brossart@brossart.com,+39-397-555-2444,"77.168,00 €",1941-03-19,F,False,41148
9,,D812,verbali,ZB199 AS,Via Bell'italia,37060,CN,,SAN COLOMBANO AL LAMBRO,,...,Osaretin,,mille (mila),dorothy@cox.net,+39-368-555-423,"93.928,00 €",1961-09-13,femmina,false,41149


In [9]:
pd.DataFrame(y)

,CodiceFiscale,CodiceCatasto,Document,Plate,Address,CAP,ProvinceCode,Region,Municipality,Year,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId
0,IVA,NaN,Document,Plate,Address,CAP,ProvinceCode,NaN,Municipality,Year,...,Name,Surname,String,EMail,PhoneNumber,NaN,Date,BiologicalSex,Boolean,NumericId
1,IVA,CodiceCatasto,NaN,Plate,Address,CAP,ProvinceCode,NaN,Municipality,NaN,...,Name,NaN,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NaN
2,CodiceFiscale,CodiceCatasto,Document,Plate,Address,CAP,ProvinceCode,Region,Municipality,NaN,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,NaN,NaN,NumericId
3,IVA,CodiceCatasto,Document,Plate,NaN,CAP,ProvinceCode,Region,Municipality,Year,...,Name,NaN,String,EMail,PhoneNumber,NaN,Date,BiologicalSex,Boolean,NumericId
4,NaN,CodiceCatasto,Document,NaN,Address,NaN,ProvinceCode,NaN,Municipality,Year,...,Name,Surname,String,EMail,PhoneNumber,Currency,NaN,BiologicalSex,NaN,NumericId
5,IVA,CodiceCatasto,Document,Plate,Address,CAP,ProvinceCode,Region,NaN,Year,...,Name,NaN,String,EMail,PhoneNumber,Currency,NaN,BiologicalSex,Boolean,NumericId
6,IVA,CodiceCatasto,Document,NaN,Address,CAP,ProvinceCode,NaN,Municipality,Year,...,Name,Surname,String,EMail,NaN,Currency,NaN,NaN,Boolean,NumericId
7,CodiceFiscale,CodiceCatasto,Document,NaN,Address,CAP,ProvinceCode,Region,NaN,Year,...,Name,Surname,String,EMail,PhoneNumber,NaN,Date,NaN,Boolean,NaN
8,CodiceFiscale,CodiceCatasto,NaN,Plate,Address,NaN,NaN,Region,Municipality,NaN,...,Name,Surname,NaN,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId
9,NaN,CodiceCatasto,Document,Plate,Address,CAP,ProvinceCode,NaN,Municipality,NaN,...,Name,NaN,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId


In [10]:
model.predict_dataframe(X)

,CodiceFiscale,CodiceCatasto,Document,Plate,Address,CAP,ProvinceCode,Region,Municipality,Year,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId
0,IVA,NaN,Document,Plate,Address,CAP,ProvinceCode,NaN,Municipality,Year,...,Name,Surname,String,EMail,PhoneNumber,NaN,Date,BiologicalSex,Boolean,NumericId
1,IVA,CodiceCatasto,NaN,Plate,Address,CAP,ProvinceCode,NaN,Municipality,NaN,...,Name,NaN,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NaN
2,CodiceFiscale,CodiceCatasto,Document,Plate,Address,CAP,ProvinceCode,Region,Municipality,NaN,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,NaN,NaN,NumericId
3,IVA,CodiceCatasto,Document,Plate,NaN,CAP,ProvinceCode,Region,Municipality,Year,...,Name,NaN,String,EMail,PhoneNumber,NaN,Date,BiologicalSex,Boolean,NumericId
4,NaN,CodiceCatasto,Document,NaN,Address,NaN,ProvinceCode,NaN,Municipality,Year,...,Name,Surname,String,EMail,PhoneNumber,Currency,NaN,BiologicalSex,NaN,NumericId
5,IVA,CodiceCatasto,Document,Plate,Address,CAP,ProvinceCode,Region,NaN,Year,...,Name,NaN,String,EMail,PhoneNumber,Currency,NaN,BiologicalSex,Boolean,NumericId
6,IVA,CodiceCatasto,Document,NaN,Address,CAP,ProvinceCode,NaN,Municipality,Year,...,Name,Surname,String,EMail,NaN,Currency,NaN,NaN,Boolean,NumericId
7,CodiceFiscale,CodiceCatasto,Document,NaN,Address,CAP,ProvinceCode,Region,NaN,Year,...,Name,Surname,String,EMail,PhoneNumber,NaN,Date,NaN,Boolean,NaN
8,CodiceFiscale,CodiceCatasto,NaN,Plate,Address,NaN,NaN,Region,Municipality,NaN,...,Name,Surname,NaN,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId
9,NaN,CodiceCatasto,Document,Plate,Address,CAP,ProvinceCode,NaN,Municipality,NaN,...,Name,NaN,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId


In [11]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('NaN', 'CountryCode'): 613,
         ('Surname', 'String'): 173,
         ('NaN', 'Boolean'): 535,
         ('Name', 'String'): 10,
         ('Boolean', 'Surname'): 2,
         ('Boolean', 'ProvinceCode'): 3,
         ('CountryCode', 'Name'): 2,
         ('CountryCode', 'ProvinceCode'): 4})